# CP-SAT: Non-linear functions

In [ ]:
from ortools.sat.python import cp_model

import numpy as np

FLOAT_APPROX_PRECISION = 100

In [ ]:
from utils import create_boolean_is_equal_to

## CP-SAT doesn't want you to use non-linear functions

CP SAT supports many integer operations. But you can't compose an IntVar by a custom function, in particular if it's non-linear. 

For example, you can't do exp(var).

In [ ]:
from math import exp

model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
result = model.NewIntVar(0, 100 * 100, "result")

# Let's say we want result == exp(x)
# This will fail (rightfully)
model.Add(result == exp(x))

TypeError: must be real number, not IntVar

## Precomputing the function values

Taylor series are cool, but a bit of an overkill. Especially at this coarse level of precision (`10E-2`) and for a function with such well-known values. 

Instead of approximating exp, we could simply precompute its values, and use some kind of hashmap to refer to it.

In [ ]:
# TODO : Make this take any function value 
# TODO : Make this work with variable bounds

def lookup_table_exp_of_x(
    model: cp_model.CpModel,
    var: cp_model.IntVar,
    float_precision_var=FLOAT_APPROX_PRECISION,
    float_precision_exp=FLOAT_APPROX_PRECISION,
):
    lb = var.Proto().domain[0]
    ub = var.Proto().domain[1]
    x_to_exp_x = {
        x: round(exp(x / float_precision_var) * float_precision_exp)
        for x in np.arange(lb, ub + 1)
    }
    exp_of_var = model.NewIntVar(
        min(x_to_exp_x.values()), max(x_to_exp_x.values()), f"exp_{var.Name()}"
    )

    # This is how we implement a kind of lookup table
    for x_value, exp_value in x_to_exp_x.items():
        var_is_equal_to_x = create_boolean_is_equal_to(model, var, x_value)
        model.Add(exp_of_var == exp_value).OnlyEnforceIf(var_is_equal_to_x)
    
    return exp_of_var

Example with the exponential function

In [ ]:
float_precision_var = 100
float_precision_exp = 10_000
value_of_x = 0.69

model = cp_model.CpModel()

x = model.NewIntVar(-10 * float_precision_var, 10 * float_precision_var, "x")
exp_of_x = lookup_table_exp_of_x(
    model,
    x,
    float_precision_var=float_precision_var,
    float_precision_exp=float_precision_exp,
)

model.Add(x == round(float_precision_var * value_of_x))

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

print(
    f"Solver value of exp({value_of_x}) is: {solver.Value(exp_of_x)/ float_precision_exp}"
)
print(f"Real value is: {exp(value_of_x)}")


Status = OPTIMAL
Solver value of exp(0.69) is: 1.9937
Real value is: 1.9937155332430823


/tmp/ipykernel_4537/1959322448.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  model.Add(value != var).OnlyEnforceIf(boolean_var.Not())
